<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Store Sales Forecasting with In-Database Time Series </b>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial'>Good Eats Grocery is a renowned retail corporation that operates a chain of hypermarkets. Here, Good Eats Grocery has provided a data combining of 45 stores including store information and monthly sales. The data is provided on weekly basis. Good Eats Grocery tries to find the impact of holidays on the sales of store. For which it has included four holidays’ weeks into the dataset which are Christmas, Thanksgiving, Super bowl, Labour Day. Here we are owing to Analyze the dataset given.<br>
<br>  
Our Main Objective is to predict sales of store in a week. As in dataset size and time related data are given as feature, so analyze if sales are impacted by time-based factors and space- based factor. Most importantly how inclusion of holidays in a week soars the sales in store?
<br>
<p style = 'font-size:16px;font-family:Arial'>Unbounded Array Framework (UAF) is the Teradata framework for building end-to-end time series forecasting pipelines. It also provides functions for digital signal processing and 4D spatial analytics. The series can reside in any Teradata supported or Teradata accessible table or in an analytic result table (ART).</p>

<p style = 'font-size:16px;font-family:Arial'>UAF provides data scientists with the tools for all phases of forecasting:</p>
<li style = 'font-size:16px;font-family:Arial'>Data preparation functions</li>
<li style = 'font-size:16px;font-family:Arial'>Data exploration functions</li>
<li style = 'font-size:16px;font-family:Arial'>Model coefficient estimation functions</li>
<li style = 'font-size:16px;font-family:Arial'>Model validation functions</li>
<li style = 'font-size:16px;font-family:Arial'>Model scoring functions</li>

<p></p>    
<br>  
<p style = 'font-size:16px;font-family:Arial'>Hence as a data science consultant, we are showcasing the complete approach about how we can make prediction of weekly sales for different stores in advance. We are demonstrating how we can train our models and use them for scoring using the ClearScape Analytics platform. The data we are using is a sample dataset and the results and predictions may not be entirely accurate.
</p>



<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>1. Data</b></p>
<p style = 'font-size:16px;font-family:Arial'>The dataset contains historical sales data for 45 Good Eats Grocery stores located in different regions. Each store contains a number of departments, and you are tasked with predicting the department-wide sales for each store.</p>

<p style = 'font-size:16px;font-family:Arial'>In addition, Good Eats Grocery runs several promotional markdown events throughout the year. These markdowns precede prominent holidays, the four largest of which are the Super Bowl, Labour Day, Thanksgiving, and Christmas. The weeks including these holidays are weighted five times higher in the evaluation than non-holiday weeks. Part of the challenge presented by this competition is modelling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data.</p>

<p style = 'font-size:16px;font-family:Arial'>The basic idea of analyzing the Good Eats Grocery Forecasting dataset is to get a fair idea about the factors affecting the Sales of the Good Eats Grocery Store.</p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>2. Start by connecting to the Vantage system.</b></p>

<p style = 'font-size:16px;font-family:Arial'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
import time
from teradataml import * 
from teradataml.context.context import *
from teradataml.dataframe.dataframe import DataFrame

from teradataml.dataframe.copy_to import copy_to_sql
import getpass
import warnings
warnings.filterwarnings('ignore')

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
eng.execute('''SET query_band='DEMO=SalesForecastARIMA.ipynb;' UPDATE FOR SESSION; ''')

<hr>
<b style = 'font-size:20px;font-family:Arial;color:#E37C4D'>2. Getting Data for This Demo </b>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string. 


In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_SalesForecasting_cloud');"
 # Takes about 45 seconds
#%run -i ../run_procedure.py "call get_data('DEMO_SalesForecasting_local');"
 # Takes about 70 seconds

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Prepare data to do some basic Analysis of the Sales data.</b></p>


In [ ]:
query1='''
SELECT
  Store, 
  Dept,
  Case When IsHoliday = 0 Then 'False' else 'True' end as IsHoliday,
  extract(year from Sales_Date) as Sales_Year,
  CAST("Sales_Date" as Timestamp) as "Sales_Date",
  Weekly_Sales
  FROM
  DEMO_SalesForecasting.Weekly_Sales
  ;
  '''
testdf=DataFrame.from_query(query1).to_pandas()

In [ ]:
testdf.head()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b> Let's do some basic analysis of the dataset </b></p>
<p style = 'font-size:16px;font-family:Arial'>We group the weekly sales by Sales Date. Will provide Average Sales based on Sales date.</p>

In [ ]:
test_df1 = testdf[['Sales_Date','Weekly_Sales']].groupby('Sales_Date').mean().reset_index()

In [ ]:
test_df1.head(5)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
plt.figure(figsize=(20,12))
plt.plot(test_df1['Sales_Date'],test_df1['Weekly_Sales'])
plt.title('Average Weekly Sales', fontsize=20)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Week', fontsize=16)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the Average Sales per week. We can see that there are peaks mainly during the Year end period.</p>
<p style = 'font-size:16px;font-family:Arial'>Next we try to get the average sales for each Store, for that we group the Weekly Sales by each Store.</p>

In [ ]:
weekly_sales = testdf[['Store','Weekly_Sales']].groupby('Store').mean().reset_index()
plt.figure(figsize=(20,12))
sns.barplot(weekly_sales['Store'], weekly_sales['Weekly_Sales'], palette='dark')
plt.grid()
plt.title('Average Sales per Store', fontsize=20)
plt.xlabel('Store', fontsize=16)
plt.ylabel('Sales', fontsize=16)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the Average Weekly Sales for each store. We can see that Store 4 shows highest weekly sales while Store 5 shows the lowest weekly sales.</p>
<p style = 'font-size:16px;font-family:Arial'>Next we try to get the Weekly Sales for each year separately. For this we group the data for all 3 years by Sales Date for each year</p>

In [ ]:
test_df2 = testdf[['Sales_Year','Sales_Date','Weekly_Sales']].groupby('Sales_Date').mean().reset_index()

In [ ]:
test_df2.head()

In [ ]:
import numpy as np
# Generate some sample data for the lines
Sales_Week = test_df2['Sales_Date'].dt.week
x = Sales_Week
# Put array of years here
y1 = test_df2[test_df2['Sales_Year']==2010]['Weekly_Sales'] # Put store1 data of sales(year wise)
y2 = test_df2[test_df2['Sales_Year']==2011]['Weekly_Sales'] # Put store2 data of sales(year wise)
y3 = test_df2[test_df2['Sales_Year']==2012]['Weekly_Sales'] # Put store3 data of sales(year wise)

In [ ]:
plt.figure(figsize=(20,8))
sns.lineplot(x,y1)
sns.lineplot(x,y2)
sns.lineplot(x,y3)
plt.grid()
plt.xticks(np.arange(1,60, step=1))
plt.legend(['2010', '2011', '2012'], loc='best', fontsize=16)
plt.title('Average Weekly Sales Per Year', fontsize=20)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Week', fontsize=16)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the Average Weekly Sales for different years. We can see that there are peaks mainly during the Year end period(47th and 51st week) for both years(2010 and 2011).</p>
<p style = 'font-size:16px;font-family:Arial'>We try to get the comparison of Sales during Holidays and Other Working Days. We do a grouping of data for Sales based on whether the Sale is on Holiday or Working Day</p>

In [ ]:
#weekly_sales_holiday = testdf[['Sales_Date','IsHoliday','Weekly_Sales']].groupby('Sales_Date','IsHoliday').mean().reset_index()
weekly_sales_holiday=testdf.groupby(['Sales_Date','IsHoliday']).agg({'Weekly_Sales':['sum','mean']})

In [ ]:
weekly_sales_holiday.columns = ['Weekly_Sales_mean', 'Weekly_Sales_sum']
weekly_sales_holiday = weekly_sales_holiday.reset_index()
weekly_sales_holiday.head()

In [ ]:
plt.figure(figsize=(20,12))
Sales_Week = test_df2['Sales_Date'].dt.week
#sns.barplot(Sales_Week, weekly_sales_holiday['Weekly_Sales_sum'], palette='dark')
sns.barplot(x=Sales_Week, y='Weekly_Sales_sum', hue='IsHoliday', data=weekly_sales_holiday, palette='dark')
plt.grid()
plt.title('Total Sales per Week', fontsize=20)
plt.xlabel('Week', fontsize=16)
plt.ylabel('Total Weekly Sales', fontsize=16)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the Weekly Sales per Week. The Orange coloured bars show weekly sales during Holidays while the Blue coloured bars show weekly sales during working days.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Preparing Dataset by joining the datasets.</b></p>

<p style = 'font-size:16px;font-family:Arial'>
    <li style = 'font-size:16px;font-family:Arial'>Weekly_Sales is our variable of interest. </li>
    <li style = 'font-size:16px;font-family:Arial'>Type, Size, Temperature, isHoliday, Fuel_Price, MarkDown1, MarkDown2, MarkDown3, MarkDown4, MarkDown4 are exogenous variables.</li>
</p>

<p style = 'font-size:16px;font-family:Arial'>We prepare the dataset by creating a view by joining data from Weekly Sales, Stores and features.</p>

In [ ]:
query2='''REPLACE VIEW Weekly_Sales_Details as select w.Sales_date  as times, CAST('2012-02-03'as DATE ) as cutoff_date,w.Dept,w.Store,
CAST(w.Sales_Date as Timestamp) as Sales_Date,ZEROIFNULL(Weekly_Sales) as Weekly_Sales,
ZEROIFNULL(Store_Size) as Store_Size ,Store_Type as Store_Type, w.IsHoliday,
ZEROIFNULL(Temperature) as Temperature,ZEROIFNULL(MarkDown1) as MarkDown1,
ZEROIFNULL(MarkDown2) as MarkDown2,ZEROIFNULL(MarkDown3) as MarkDown3,ZEROIFNULL(MarkDown4) as MarkDown4,ZEROIFNULL(MarkDown5) as MarkDown5,
ZEROIFNULL(CPI) as CPI,ZEROIFNULL(Unemployment) as Unemployment,ZEROIFNULL(Fuel_Price) as Fuel_Price , cast(trim(w.Dept)||trim(w.Store) as INT)  as idcols
from Demo_SalesForecasting.Weekly_Sales w 
LEFT JOIN Demo_SalesForecasting.Stores s
ON w.Store = s.Store
LEFT JOIN Demo_SalesForecasting.Features f
ON w.Store = f.store
and w.Sales_Date = f.Sales_Date
where w.Store in (20,4);'''

eng.execute(query2)
modeldf=DataFrame.from_query('select * from Weekly_Sales_Details;')

In [ ]:
dfacheck = modeldf.groupby(["idcols"])
dfacheck=dfacheck.count().select(["idcols","count_Sales_Date"])

dfa4=modeldf.join(dfacheck, on = 'idcols', how = "left", lsuffix = 't1', rsuffix = 't2').drop(['t2_idcols'],axis=1)
dfa4=dfa4.assign(idcols = dfa4['t1_idcols'])
dfa4=dfa4.drop(['t1_idcols'],axis=1)

# filter out incomplete time series 

modeldf1 = dfa4[dfa4.count_Sales_Date == 143]
modeldf1.shape

In [ ]:
modeldf1.head()

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>3. Checking for Stationarity of Time Series using the Dickey Fuller Test</b></p>

<p style = 'font-size:16px;font-family:Arial'>To be able to model a time series, it needs to be stationary. ARIMA models deal with non-stationary time series by differencing (The "d' parameter in ARIMA determines the number of differences needed to make a series stationary)</p>
<p style = 'font-size:16px;font-family:Arial'>Here we will check for stationarity of all time series using the Dickey-Fuller Test. For more info on the test,  see <a href="https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference-17.20/Diagnostic-Statistical-Test-Functions/TD_DICKEY_FULLER/TD_DICKEY_FULLER-Example">here.</a> 
<p style = 'font-size:16px;font-family:Arial'>The null hypothesis for the test is that the data is non-stationary. We want to REJECT the null hypothesis for this test. So we want a p-value of less that 0.05 (or smaller) and a negative coefficient value for the lag term in our regression model.</p> 
<p style = 'font-size:16px;font-family:Arial'>The Dickey fuller function needs series data so we use the TDSeries function to create a series and apply DickeyFuller to check the stationarity of the data.</p>
<p style = 'font-size:16px;font-family:Arial'>We use the OutlierFilterFit and the OutlierFilterTransform functions to remove the outliers in the series and then use the Rescaled Data to check the stationarity of the data using the DickeyFuller function.</p>


In [ ]:
sales_df=modeldf1

In [ ]:
sales_df.shape

<p style = 'font-size:16px;font-family:Arial'>The OutlierFilterFit() function calculates the lower_percentile, upper_percentile, count of rows and median for all the "target_columns" provided by the user. These metrics for each column helps the function OutlierTransform() detect outliers in the input table. It also stores parameters from arguments into a FIT table used during   transformation.</p>


In [ ]:
from teradataml import OutlierFilterFit
OutlierFilterFit_out = OutlierFilterFit(data = sales_df,
                                            target_columns = "Weekly_Sales",
                                               )
out_df=OutlierFilterFit_out.output_data
out_df

<p></p>
<p style = 'font-size:16px;font-family:Arial'> The OutlierFilterfit creates a fit table with different values which need to be applied on the data to get the transformed data.</p>
<p></p>
<p style = 'font-size:16px;font-family:Arial'> OutlierFilterTransform() function filters the outliers from the input teradataml DataFrame.</p> 
<p style = 'font-size:16px;font-family:Arial'>OutlierFilterTransform() uses the result DataFrame from OutlierFilterFit() function to get statistics like median, count of rows, lower percentile and upper percentile for every column specified in target columns argument and filters the outliers in the input data. </p>

In [ ]:
from teradataml import OutlierFilterFit, OutlierFilterTransform
obj = OutlierFilterTransform(data=sales_df,
                                 object=OutlierFilterFit_out.result)
out_transform_df = obj.result
out_transform_df.head(5)

<p></p>
<p style = 'font-size:16px;font-family:Arial'> The OutlierFilterTransform transforms the data and creates the output data after applying the Fit Table details on the data.</p>
<p></p>
<p style = 'font-size:16px;font-family:Arial'>The Resample() function transforms an irregular time series into a regular time series. It can also be used to alter the sampling interval for a time series.</p>

In [ ]:
from teradataml import Resample
data_series_df = TDSeries(data=obj.result,
                              id="idcols",
                              row_index=("Sales_Date"),
                              row_index_style= "TIMECODE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL")

In [ ]:
uaf_out1 = Resample(data=data_series_df,
                        interpolate='LINEAR',
                        timecode_start_value="TIMESTAMP '2010-02-05 00:00:00'",
                        timecode_duration="WEEKS(1)")

In [ ]:
df=uaf_out1.result
df1=df.select(['idcols','ROW_I', 'Weekly_Sales']).assign(Sales_Date=df.ROW_I)
df1

<p></p>
<p style = 'font-size:16px;font-family:Arial'>The DickeyFuller() function tests for the presence of one or more unit roots in a series to determine if the series is non-stationary. When a series contains unit roots, it is non-stationary. When a series contains no unit roots, whether the series is stationary is based on other factors.</p>
<p style = 'font-size:16px;font-family:Arial'>

In [ ]:
from teradataml import DickeyFuller
data_series_df_1 = TDSeries(data=df1,
                              id="Sales_Date",
                              row_index=("idcols"),
                              row_index_style= "SEQUENCE",
                              payload_field="Weekly_Sales",
                              payload_content="REAL")

In [ ]:
df_out = DickeyFuller(   data=data_series_df_1,
                           algorithm='NONE')

# Print the result DataFrame.
print(df_out.result)

<p></p>
<p style = 'font-size:16px;font-family:Arial'>In the above output the p-value corresponding to the calculated test statistic is less than 0.05. It means that the series is stationary. The output column NULL_HYP which means NULL HYPOTHESIS can have 2 values 
    <li style = 'font-size:16px;font-family:Arial'>ACCEPT means the null hypothesis is accepted. No Unit roots are present, and therefore the process is stationary.</li>
<li style = 'font-size:16px;font-family:Arial'>REJECT means the null hypothesis is rejected. Unit roots are present, and the process may or may not be stationary, depending on other factors.</li>
</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>4. ARIMA Modelling</b></p>

<p style = 'font-size:16px;font-family:Arial'>ARIMA is a statistical model for analysing and forecasting time series data. It adds the parts of integration to regression for making the data stationary with the use of differentiation. ARIMA is a notation for (p,d,q) where the parameters are filled with values to indicate the ARIMA model being used for the purpose of training, testing and forecasting.</p>
<p style = 'font-size:16px;font-family:Arial'>The ArimaEstimate() function estimates the coefficients corresponding to an ARIMA (AutoRegressive Integrated Moving Average) model, and to fit a series with an existing ARIMA model. The function can also provide the "goodness of fit" and the residuals of the fitting operation. The function generates model layer used as input for the ArimaValidate() and ArimaForecast() functions. This function is for univariate series.</p>

<p style = 'font-size:16px;font-family:Arial'>The following procedure is an example of how to use ArimaEstimate() function:</p>
<li style = 'font-size:16px;font-family:Arial'> Run the ArimaEstimate() function to get the coefficients for the ARIMA model.
<li style = 'font-size:16px;font-family:Arial'> [Optional] Run ArimaValidate() function to validate the 'goodness of fit' of the ARIMA model, when "fit_percentage" argument value is not 100 in ArimaEstimate() function.
<li style = 'font-size:16px;font-family:Arial'>Run the ArimaForecast() function with input from step 1 or step 2 to forecast the future periods beyond the last observed period.</li>
</p>

<p style = 'font-size:16px;font-family:Arial'> Here the input series to the ArimaEstimate is the output series of the Resample function. The series is created by using the output of Resample function and passed to ArimaEstimate. </p>

In [ ]:
from teradataml import ArimaEstimate
# Execute ArimaEstimate function.
arima_est_out = ArimaEstimate(data1=data_series_df_1,
                            nonseasonal_model_order=[2,1,1],
                            constant=False,
                            algorithm="MLE",
                            coeff_stats=True,
                            fit_metrics=True,
                            residuals=True,
                            fit_percentage=80)

In [ ]:
est_result=arima_est_out.fitresiduals
est_result_sql=est_result.to_pandas().reset_index()
est_result_sql1=est_result_sql[est_result_sql['ROW_I'] == 22].head()
est_result_sql1.head()

<p style = 'font-size:16px;font-family:Arial'>We plot the Actual Value of Weekly Sales vs the Calculated Value of the ArimaEstimate function. </p>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
# x = est_result_sql[est_result_sql['Store']==20]['Sales_Date']
x = est_result_sql['Sales_Date']
# Put array of years here
y1 = est_result_sql['ACTUAL_VALUE']
#y1=est_result_sql[est_result_sql['Store']==20]['ACTUAL_VALUE']
y2 = est_result_sql['CALC_VALUE']
plt.figure(figsize=(20,8))
sns.lineplot(x,y1,ci=None)
sns.lineplot(x,y2,ci=None)
plt.grid()
plt.legend(['Actual Value', 'Predicted Value'], loc='best', fontsize=16)
plt.title('Comparison of Actual vs Estimated Sales ', fontsize=20)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Sales Date', fontsize=16)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The ArimaValidate() function performs an in-sample forecast for both seasonal and non-seasonal auto-regressive (AR), moving-average (MA), ARIMA models and Box-Jenkins seasonal ARIMA model formula followed by an analysis of the produced residuals. The aim is to provide a collection of metrics useful to select the model and expose the produced residuals such that multiple model validation and statistical tests can be conducted.</p>

In [ ]:
from teradataml import ArimaValidate
data_art_df = TDAnalyticResult(data=arima_est_out.result)


arima_val_out = ArimaValidate(data=data_art_df, fit_metrics=True, residuals=True)

In [ ]:
val_result=arima_val_out.fitresiduals
val_result_sql=est_result.to_pandas().reset_index()
val_result_sql1=val_result_sql[val_result_sql['ROW_I'] == 93].head()
val_result_sql1.head()

<p style = 'font-size:16px;font-family:Arial'>We plot the Actual Value of Weekly Sales vs the Calculated Value of the ArimaValidate function. </p>

In [ ]:
x = val_result_sql['Sales_Date']
# Put array of years here
y1 = val_result_sql['ACTUAL_VALUE']
y2 = val_result_sql['CALC_VALUE']
plt.figure(figsize=(20,8))
sns.lineplot(x,y1,ci=None)
sns.lineplot(x,y2,ci=None)
plt.grid()
# plt.xticks(np.arange(1,60, step=1))
plt.legend(['Actual Value', 'Predicted Value'], loc='best', fontsize=16)
plt.title('Comparison of Actual vs Validated Sales', fontsize=20)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Sales Date', fontsize=16)
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The ArimaForecast() function is used to forecast a user-defined number of periods based on models fitted from the ArimaEstimate() function.</p>
<p style = 'font-size:16px;font-family:Arial'>Here we are considering 7 periods (forecast_periods=7)</p>

In [ ]:
from teradataml import ArimaForecast
arima_estimate_op = ArimaEstimate(data1=data_series_df_1,
                                      nonseasonal_model_order=[2,1,1],
                                      constant=False,
                                      algorithm="MLE",
                                      coeff_stats=True,
                                      fit_metrics=True,
                                      residuals=True,
                                      fit_percentage=100)

# Create teradataml TDAnalyticResult object over the result attribute of 'arima_estimate_op'
data_art_df = TDAnalyticResult(data=arima_estimate_op.result)
 
arima_forcast_out = ArimaForecast(data=data_art_df, forecast_periods=7)

In [ ]:
forecast_result=arima_forcast_out.result
forecast_result_sql=forecast_result.to_pandas().reset_index()
forecast_result_sql=forecast_result_sql[['ROW_I','FORECAST_VALUE']]
forecast_result_sql.head()

<p style = 'font-size:16px;font-family:Arial'>We plot the Forecasted Value of Weekly Sales for the defined number of periods. </p>

In [ ]:
x = forecast_result_sql['ROW_I']
# Put array of years here
y = forecast_result_sql['FORECAST_VALUE']
plt.figure(figsize=(20,8))
sns.lineplot(x,y,ci=None)
plt.grid()
# plt.xticks(np.arange(1,60, step=1))
plt.legend(['Forecast Value'], loc='best', fontsize=16)
plt.title('Forecast Sales', fontsize=20)
plt.ylabel('Forecast Sales', fontsize=16)
plt.xlabel('Forcast Period', fontsize=16)
plt.show()

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Conclusion:</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have trained and validated the ARIMA model on the Weekly Sales dataset, and the results closely match the actual data. The goodness of fit metrics calculated in the estimate and validate phase also resonate with our understanding that the model is well-trained to forecast. This can be observed in the Estimate and the Validate function graphs. So we can say that the model is well trained to forecast the Weekly Sales.</p>

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>5. Cleanup</b></p>


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_SalesForecasting');" 
#Takes 45 seconds

In [ ]:
remove_context()

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">Copyright © Teradata Corporation - 2023. All Rights Reserved.</footer>